<a href="https://colab.research.google.com/github/marketpsych/marketpsych/blob/main/notebooks/load_data_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading CSV files directly from SFTP

This notebook shows how to load MarketPsych's data with your SFTP credentials directly into a Jupyter Notebook. Note, however, that this notebook is an **alpha version**. For more robust testing, we recommend downloading the trialing flat files, as instructed by the MRNSupport, and then loading them into your own environment. 

---
## Settings
In order to have this example working, you'll need to install MarketPsych's library. Additionally, this notebook uses some widgets to facilitate with your navigation. To install the libraries and enable widgets, please run the following cell. 

In [2]:
import sys
# Installs marketpsych library into your environment
!{sys.executable} -m pip install marketpsych --upgrade --quiet
# Installs ipywidget library into your environment
!{sys.executable} -m pip install ipywidgets --upgrade --quiet

## Libraries
from marketpsych import sftp
from marketpsych import mpwidgets

# Allows using the widgets
!{sys.executable} -m jupyter nbextension enable --py widgetsnbextension

from IPython.core.magic import register_cell_magic
from IPython.display import HTML, display


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


---
## Selecting your login credentials

Now you should be able to input your credentials. I.e., the path to the `Key file`, and `UserID` as provided by the MRNSupport. 



<font color='blue'>1. Run the following cell.</font>                      
<font color='blue'>2. Make your selections through the widgets.</font>                      



First, upload your .ppk key by using the left `Key file` button to select the file. The User ID is then guessed through the provided key. Please check if that is correct (if it is the same ID as the one provided by MRNSupported). If not, manually change the ID.

In [5]:
cwdgts = mpwidgets.LoginWidgets()
cwdgts.display()

**WARNING**                         
When you run the following cell after waiting for too long, you may be given the following error:
```python
"ValueError": I/O operation on closed file.
```   

If so, please re-run the previous cell and upload your key. Otherwise, continue.

In [6]:
# Creates client
client = sftp.connect(user=cwdgts.id_widget.value, key=cwdgts.key_widget.content)

---
## Loading the data

Finally, you can download the files directly into a pandas dataframe. The options can be defined through the 5 widgets below.

 - Select the checkbox if you are trialing the data. In some special cases, even if you are trialing, you may need to uncheck it (you can try both options in case of Permission errors).

 - The options for **Asset class** are:  

|Asset class   | Description|
|:-------------|:------------|
|`CMPNY`       | Individual companies|
|`CMPNY_AMER`  | Individual companies domiciled in America|
|`CMPNY_APAC`  | Individual companies domiciled in APAC|
|`CMPNY_EMEA`  | Individual companies domiciled in EMEA|
|`CMPNY_ESG`   | Individual companies (ESG package)|
|`CMPNY_GRP`   | Company groups and ETFs|
|`COM_AGR`     | Agricultural commodities|  
|`COM_ENM`     | Energy and Metals|
|`COU`         | Countries|
|`COU_ESG`     | Countries (ESG package)|
|`COU_MKT`     | Stock indices, sovereign bonds, real estate|
|`CRYPTO`      | Cryptocurrencies|
|`CUR`         | Currencies|  

 - The options for **frequencies** are:  
  
|Frequency  | Description| Use case |
|:----------|:-----------|:---------|
|`W365_UDAI`| Yearly lookback window and daily updates| ESG Core only |
|`WDAI_UDAI`| Daily lookback window and daily updates| Daily data stamped 30 minutes before the NYSE close|
|`WDAI_UHOU`| Daily lookback window and hourly updates| Daily data stamped hourly (in case you want daily data adjusted to your time-zone) |
|`W01M_U01M`| Minutely lookback window and minutely updates| Low-latency data (**WARNING:** extremely large datasets)|
 
- Regarding **dates**, simply select the start and end dates of interest. Note that for dates older than 2 months (with respect to your current date), the files are packaged in monthly batches. Hence, only the selection of month will matter and not the specific day. For example, if you select `2020-12-25` as the start or end date, the full `2020-12` month will be loaded anyways.

**WARNINGS**   
Loading large files such as CMPNY data with a long window-frame can take quite a while and take over your memory. Start by loading very short periods (e.g., one month of data), then moving to ever longer periods. 

Check your asset class permissions. If you try downloading data for which no access was provided, it will give a Permission error:
```python
"PermissionError": [Errno 13] Access denied
```   

<font color='blue'>Select options for loading the data after running the cell:</font> 

In [7]:
lwdgts = mpwidgets.LoaderWidgets()
lwdgts.display()

Checkbox(value=True, description='Trial')

Dropdown(description='Asset class:', index=7, options=('CMPNY', 'CMPNY_AMER', 'CMPNY_APAC', 'CMPNY_EMEA', 'CMP…

Dropdown(description='Frequency:', index=1, options=('W365_UDAI', 'WDAI_UDAI', 'WDAI_UHOU', 'W01M_U01M'), valu…

DatePicker(value=datetime.datetime(2020, 12, 1, 0, 0), description='Start date:')

DatePicker(value=datetime.datetime(2020, 12, 31, 0, 0), description='End date:')

In [8]:
%%time

df = client.download(
    asset_class=sftp.AssetClass[lwdgts.asset_class_widget.value],
    frequency=sftp.Frequency[lwdgts.frequency_widget.value],
    start=lwdgts.start_date_widget.value,
    end=lwdgts.end_date_widget.value,
    trial=lwdgts.trial_check_widget.value
)

display(df)

,id,assetCode,windowTimestamp,dataType,systemVersion,mentions,buzz,sentiment,negative,positive,...,regulatoryIssues,supplyVsDemand,supplyVsDemandForecast,acreageCultivated,agDisease,agStress,subsidies,subsidiesSentiment,weatherDamage,futureVsPast
0,mp:2020-12-01_20.30.00.News.COM_AGR.COC,COC,2020-12-01T20:30:00.000Z,News,MP:4.0.0,646,1379.2,-0.090270,0.239632,0.149362,...,NaN,0.013051,0.000000,NaN,NaN,0.005800,NaN,NaN,NaN,-0.419301
1,mp:2020-12-01_20.30.00.News_Social.COM_AGR.COC,COC,2020-12-01T20:30:00.000Z,News_Social,MP:4.0.0,716,1604.9,-0.059505,0.229921,0.170416,...,NaN,0.018693,0.007477,NaN,NaN,0.004985,NaN,NaN,NaN,-0.393981
2,mp:2020-12-01_20.30.00.News_Headline.COM_AGR.COC,COC,2020-12-01T20:30:00.000Z,News_Headline,MP:4.0.0,52,95.5,-0.109948,0.062827,-0.047120,...,NaN,0.020942,0.020942,NaN,NaN,NaN,NaN,NaN,NaN,0.172775
3,mp:2020-12-01_20.30.00.Social.COM_AGR.COC,COC,2020-12-01T20:30:00.000Z,Social,MP:4.0.0,70,225.7,0.128489,0.170580,0.299070,...,NaN,0.053168,0.053168,NaN,NaN,NaN,NaN,NaN,NaN,-0.239256
4,mp:2020-12-01_20.30.00.News.COM_AGR.COF,COF,2020-12-01T20:30:00.000Z,News,MP:4.0.0,163,483.0,0.052795,0.255694,0.308489,...,NaN,-0.008282,-0.005176,-0.001035,NaN,0.099379,NaN,NaN,0.006211,-0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,mp:2020-12-31_20.30.00.Social.COM_AGR.WHT,WHT,2020-12-31T20:30:00.000Z,Social,MP:4.0.0,161,550.1,-0.156335,0.307217,0.150882,...,NaN,0.018179,0.000909,-0.000909,0.005454,0.014543,NaN,NaN,NaN,-0.168878
2104,mp:2020-12-31_20.30.00.News.COM_AGR.WOO,WOO,2020-12-31T20:30:00.000Z,News,MP:4.0.0,1,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.500000
2105,mp:2020-12-31_20.30.00.News_Social.COM_AGR.WOO,WOO,2020-12-31T20:30:00.000Z,News_Social,MP:4.0.0,4,42.5,0.082353,0.282353,0.364706,...,NaN,NaN,NaN,0.023529,NaN,NaN,NaN,NaN,NaN,-0.529412
2106,mp:2020-12-31_20.30.00.News_Headline.COM_AGR.WOO,WOO,2020-12-31T20:30:00.000Z,News_Headline,MP:4.0.0,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


CPU times: user 149 ms, sys: 34.6 ms, total: 183 ms
Wall time: 5.62 s


If you can see a dataframe above, congratulations! You have downloaded some data into your notebook. From here, you can have fun exploring it. Below, you'll find a plotting tool for some simple understanding of the data.

---
## Visualizing the data

Below you can use the widgets to do some very basic exploration. A description of the widgets is given below.

- The **Data Type** field represents the type of content source(s) on which the RMAs are based. There are four possible values:
    - `News` for news sources (headlines and corpus)
    - `News_Headline` for the headline only of news sources
    - `Social` for social media sources
    - `News_Social` for the combined content               


- The **Anaytics** field represents the RMA. The actual values will depend on the asset class. Several types of indicators are provided:
    - Emotional indicators such as Anger, Fear and Joy
    - 'Economic' metrics including Earnings Forecast, Interest Rate Forecast, Long vs. Short 
    - ESG measures including CarbonEmissionsControversy, ManagementTrust, and WorkplaceSafety
    - etc. 


- The **Asset** field represents the asset of choice. To see all options, clear the cell. For a description of each asset, please search for the asset code in the User Guide or Eikon app.


- The **Roll. window** field represents the length in the smoothing function (a simple moving average).

The indicators are updated every minute for companies, sectors, regions, countries, commodities, indices, bonds, currencies and cryptocurrencies. They can be translated directly into spreadsheets or charts that can be monitored by traders, risk managers or analysts – or they can be plugged straight into your algorithms for low frequency or longer-term asset allocation or sector rotation decisions.

**WARNING**                
If your plot is empty, it is likely that there is no data for that combination of the three top variables. 

<font color='blue'>Select options for plotting the data after running the cell:</font> 

In [9]:
swdgts = mpwidgets.SlicerWidgets(df)
swdgts.display()